## Snazzy Processing Pipeline

Integrates all of the modules from the package, to go from raw data to csv output.
More information about each module can be found in the other Jupyter Notebooks.
You should change the `dataset_name` and the `img_path` at the fist cell.

If the raw data is in nd2 format, it can be converted directly using the pipeline.
For other formats, the image must me converted to tif using another application, for example ImageJ.

Adjust the image path to where the tif/nd2 file is.

In [ ]:
%reload_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

from snazzy_processing import pipeline, slice_img

dataset_name = '20240611'
root_dir = Path.cwd().parent
project_dir = root_dir.joinpath('data', dataset_name)
res_dir = root_dir.joinpath('results', dataset_name)
res_dir.mkdir(parents=True, exist_ok=True)

# Provide an absolute path for the raw tif or nd2 image
img_path = Path.home().joinpath("Documents", "raw_data", f"{dataset_name}.nd2")

Extract the first frames from the image. These frames are used to determine the embryo positions.

In [ ]:
first_frames = "first_frames.tif"
first_frames_path = root_dir.joinpath("results", dataset_name, first_frames)
if first_frames_path.exists():
    print(f"{first_frames_path.stem} already exists.\nPlease choose another path.")
else:
    slice_img.save_first_frames_as_tiff(img_path, first_frames_path, 10)

Visualize the regions of the individual embryos that will be cropped.
If the bounding boxes don't fit the embryo, you can try changing the `thres_adjust` parameter in `slice_img.calculate_slice_coordinates`.
Decreasing `thres_adjust`, which can be a negative value, tends to generate bigger boxes.

The resulting image also shows the number that will be used for each embryo when saving the individual movies.

If after adjust `thres_adjust` the embryos are still not framed as expected, you can use `slice_img.increase_bbox()` to control the bbox dimensions.

In [ ]:
img = slice_img.get_first_image(first_frames_path)

coords = slice_img.calculate_slice_coordinates(
    first_frames_path, n_cols=2, thres_adjust=-10
)

# If necessary, manually change the bboxes size by changing w and h:
boundaries = slice_img.increase_bbox(coords, w=190, h=40, shape=img.shape)

rect_coords = [slice_img.boundary_to_rect_coords(b) for b in boundaries.values()]
recs = [Rectangle((y, x), w, h) for (x, y, w, h) in rect_coords]

centroids = [((x0 + x1) // 2, (y0 + y1) // 2) for (x0, x1, y0, y1) in coords.values()]
h = img.shape[0]
w = img.shape[1]

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
ax.set_axis_off()
ax.imshow(img)

for i, (x0, x1, y0, y1) in boundaries.items():
    centroid_x = (x0 + x1) // 2
    centroid_y = (y0 + y1) // 2
    ax.text(centroid_y, centroid_x, str(i))

pc = PatchCollection(recs, color="red", alpha=0.7, linewidth=1, facecolor="none")
ax.add_collection(pc)

dest = root_dir.joinpath("results", dataset_name)
dest.mkdir(parents=True, exist_ok=True)
save_path = dest / "emb_numbers.png"
plt.savefig(save_path)
print(f"Saved image at {save_path}")
plt.show()

If the image above looks good, select which embryos you want to analyze, by changing the values in the `embryos` list in the next cell.
Embryos will be saved under the `data` directory, for the corresponding dataset.
To process all embryos you can just pass an empty `embryos` list.

The length and activity data will be saved in the `results` directory.

In [ ]:
# Delete the individual movies created after the end of the analysis
clean_up_data = False
# List of the ids of the embryos that should be processed
# The ids are the bbox numbers from the previous cell output
embryos = []
# Interval (number of frames) used to calculate VNC length
vnc_length_interval = 10
# Window (number of frames) to calculate VNC ROI (which is then used to calculate activity)
window = 1

# directories and file paths
embs_dest = root_dir.joinpath("data", dataset_name, "embs")
embs_dest.mkdir(parents=True, exist_ok=True)

# nd2 to tif
if img_path.suffix == ".nd2":
    print("Converting from nd2 to tif")
    output_path = img_path.parent.joinpath(f"{img_path.stem}.tif")
    slice_img.save_as_tiff(img_path, output_path)
    img_path = output_path

# tif to individual movies
print("Cropping individual movies")
slice_img.cut_movies(boundaries, img_path, embs_dest, embryos=embryos, overwrite=False)

# movies to length
print("Calculating VNC lengths")
num_saved_length_files = pipeline.measure_vnc_length(
    embs_dest, res_dir, vnc_length_interval
)
if num_saved_length_files > 0:
    print(f"Wrote length data for {num_saved_length_files} embryos.")

# movies to full length
print("Calculating embryo full length")
# Change low_non_VNC to True if values in the VNC periphery are very low
low_non_VNC = False
num_full_lengths_measured = pipeline.measure_embryo_full_length(
    embs_dest, res_dir, low_non_VNC=low_non_VNC
)
if num_full_lengths_measured > 0:
    print(f"Wrote full length data for {num_full_lengths_measured} embryos.")

# movies to activity
print("Calculating signal traces")
num_act_calculated = pipeline.calc_activities(embs_dest, res_dir, window)
if num_act_calculated > 0:
    print(f"Wrote activity data for {num_act_calculated} embryos.")

# delete individual movies
if clean_up_data:
    tif_path = Path(img_path)
    pipeline.clean_up_files(embs_dest, first_frames_path, tif_path)
else:
    pipeline.clean_up_files(None, first_frames_path, None)

# write params used
output_path = root_dir.joinpath("results", dataset_name, "params.txt")
pipeline.log_params(
    output_path=output_path,
    dataset_name=dataset_name,
    embryos=embryos,
    window=window,
    vnc_length_interval=vnc_length_interval,
    low_non_VNC=low_non_VNC,
)